In [26]:
!pip install openai
!pip install pinecone-client

In [27]:
from openai import OpenAI
from pinecone import Pinecone,PodSpec
import pandas as pd
from uuid import uuid4
import numpy as np

In [28]:
client= OpenAI(api_key="credential")
pc=Pinecone(api_key="credential")

In [29]:
#pc.create_index("sephora",dimension=1536,metric="cosine",spec=PodSpec(environment="gcp-starter"))

In [30]:
# Accessing the Pinecone index named "sephora"
index = pc.Index("sephora")
# Describing and retrieving statistics about the "sephora" index
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.08494,
 'namespaces': {'sephora': {'vector_count': 8494}},
 'total_vector_count': 8494}

In [31]:
# Reading product information from a CSV file into a Pandas DataFrame
sephora_products = pd.read_csv('product_info.csv')

# Converting the 'rating' column to string type
sephora_products['rating'] = sephora_products['rating'].astype(str)
sephora_products['price_usd'] = sephora_products['price_usd'].astype(str)

import re

# Define a function to clean special characters from a text
def clean_text(text):
    if isinstance(text, str):
        return re.sub(r'[^A-Za-z0-9\s]+', '', text)
    return text

# List of columns to clean
text_columns = [
    'product_name', 'size', 'ingredients',
    'highlights', 'primary_category', 'secondary_category', 'tertiary_category'
]

# Apply the cleaning function to the selected columns
for column in text_columns:
    if column in sephora_products.columns:
        sephora_products[column] = sephora_products[column].apply(clean_text)

In [32]:
sephora_products.shape[0]

8494

In [33]:
# Creating a new column 'content' in the DataFrame 'sephora_products'
# This column product details into a single string
sephora_products['product_id'].fillna('sin info', inplace=True)
sephora_products['product_name'].fillna('sin info', inplace=True)
sephora_products['highlights'].fillna('sin info', inplace=True)
sephora_products['primary_category'].fillna('sin info', inplace=True)
sephora_products['secondary_category'].fillna('sin info', inplace=True)
sephora_products['tertiary_category'].fillna('sin info', inplace=True)
sephora_products['rating'].fillna('sin info', inplace=True)
sephora_products['size'].fillna('sin info', inplace=True)
sephora_products['price_usd'].fillna('sin info', inplace=True)
sephora_products['ingredients'].fillna('sin info', inplace=True)

sephora_products["content"] = (
    "ID:(" + sephora_products["product_id"]
    + "), Name:(" + sephora_products["product_name"]
    + "), highlights:(" + sephora_products["highlights"]
    + "), primary_category:(" + sephora_products["primary_category"]
    + "), secondary_category:(" + sephora_products["secondary_category"]
    + "), tertiary_category:(" + sephora_products["tertiary_category"]
    + "), rating:(" + sephora_products["rating"]
    + "), size:(" + sephora_products["size"]
    + "), price:(" + sephora_products["price_usd"]
    + "), ingredients:(" + sephora_products["ingredients"] + ")")

In [34]:
# Crear una máscara booleana para identificar las filas con NaN en la columna 'content'
nan_content_mask = sephora_products['content'].isna()

# Filtrar el DataFrame para mostrar solo las filas con NaN en la columna 'content'
rows_with_nan_content = sephora_products[nan_content_mask]

# Mostrar las filas con NaN en la columna 'content'
print(rows_with_nan_content)



Empty DataFrame
Columns: [product_id, product_name, brand_id, brand_name, loves_count, rating, reviews, size, variation_type, variation_value, variation_desc, ingredients, price_usd, value_price_usd, sale_price_usd, limited_edition, new, online_only, out_of_stock, sephora_exclusive, highlights, primary_category, secondary_category, tertiary_category, child_count, child_max_price, child_min_price, content]
Index: []

[0 rows x 28 columns]


In [35]:
batch_limit = 10  # The maximum number of items to process in a single batch
namespace = "sephora"  # The namespace for categorizing data within the Pinecone index

In [36]:
sephora_products['content'].iloc[102]  # check the content

'ID:(P474809), Name:(Blue Algae Vitamin C Skinclarity Brightening Serum), highlights:(sin info), primary_category:(Skincare), secondary_category:(Treatments), tertiary_category:(Face Serums), rating:(4.6087), size:(1 oz 30 mL), price:(72.0), ingredients:(Water Aqua Eau CaprylicCapric Triglyceride Propanediol Glycerin Butylene Glycol Ascorbic Acid Triheptanoin Pseudoalteromonas Ferment Extract C1315 Alkane Parachlorella Beijerinckii Exopolysaccharides Biosaccharide Gum1 Bakuchiol Beta Vulgaris Root Extract Hydrolyzed Corn Starch Sodium Hyaluronate Melia Azadirachta Leaf Extract Melia Azadirachta Flower Extract Corallina Officinalis Extract Coccinia Indica Fruit Extract Solanum Melongena Fruit Extract Aloe Barbadensis Flower Extract Simmondsia Chinensis Jojoba Seed Oil Curcuma Longa Root Extract Ocimum Basilicum FlowerLeaf Extract Ocimum Sanctum Leaf Extract Salicylic Acid Tocopherol Sodium Polyacryloyldimethyl Taurate Sodium Citrate Phycocyanin Lavandula Angustifolia Oil Tanacetum Vulga

In [37]:
sephora_products.shape[0] # check number of row

8494

In [38]:
# Iterating through batches of the 'sephora_products' DataFrame, ingesting documents to Pinecone index
#for batch in np.array_split(sephora_products, len(sephora_products)/batch_limit):
#  metadata=[{"product_id":row["product_id"],
#             "content":row["content"]} for _,row in batch.iterrows()] # Creating metadata for each product in the batch
#  texts=batch["content"].tolist()
#  ids=[str(uuid4()) for _ in range(len(texts))]  # Generating unique IDs for each text using uuid4
#  response = client.embeddings.create(input=texts, model='text-embedding-3-small')
#  embeds=[np.array(x.embedding) for x in response.data]
#  index.upsert(vectors=zip(ids,embeds, metadata),namespace=namespace)




In [39]:
# Describing and retrieving statistics about the "sephora" index again to se if load the correct amount of rows
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.08494,
 'namespaces': {'sephora': {'vector_count': 8494}},
 'total_vector_count': 8494}

In [40]:
def retrieve(query, top_k,namespace,emb_model):
  query_response=client.embeddings.create(input=query,model=emb_model)  # Generate an embedding for the query using the specified embedding model
  query_emb=query_response.data[0].embedding
  retrieved_docs=[]
  # Vector search on the index using the query embedding
  docs=index.query(vector=query_emb,
                   top_k=top_k,
                   namespace=namespace,
                   include_metadata=True)
  # Iterate over the matched documents and extract their content
  for doc in docs['matches']:
    retrieved_docs.append(doc['metadata']['content'])
  return retrieved_docs

In [41]:
#check the retrieved_docs
#documents=retrieve(query,top_k=3,namespace='sephora', emb_model='text-embedding-3-small')
#documents

In [42]:
def prompt_with_context_builder(query, docs, maxlen=9500):
  delim = '\n\n---\n\n'

  # Instructs the model to answer based on the context
  prompt_start = 'Answer the question based on the context below.\n\ncontext:\n'

  # Includes the question and the placeholder for the answer
  prompt_end = f'\n\nQuestion: {query}\nAnswer:'

  # Initialize total_length to keep track of the accumulated length of the documents
  # Initialize i to keep track of the index of the last document included in the prompt
  total_length, i = 0, 0

  # Iterate over the documents
  for i, doc in enumerate(docs):
    # Add the length of the current document and the delimiter to the total length
    total_length += len(doc) + len(delim)

    # If the accumulated length exceeds the maximum length, stop adding more documents
    if total_length >= maxlen:
      break

  # Construct the prompt by concatenating the start of the prompt, the selected documents joined by the delimiter, and the end of the prompt
  prompt = prompt_start + delim.join(docs[0:(i+1)]) + prompt_end

  # Return the constructed prompt
  return prompt


In [43]:
from multiprocessing import context

In [44]:
def question_answering(prompt, sources, chat_model):
  # Define the system prompt which sets the behavior of the assistant
  sys_prompt = (
    "You are an assistant that helps a customer with answering questions. "
    "Please answer the question based on the provided context below. "
    "Make sure not to make any changes to the context, if possible, when preparing answers to provide accurate responses. "
    "If the answer cannot be found in context, just politely say that you do not know, do not try to make up an answer."
  )

  # Call the chat completion API with the specified model and messages
  res = client.chat.completions.create(
    model = chat_model,    # Use the specified chat model
    messages = [           # List of messages to send to the model
      {"role": "system", "content": sys_prompt},  # System prompt setting the behavior
      {"role": "user", "content": prompt}         # User prompt containing the question and context
    ],
    temperature = 0        # Set temperature to 0 for deterministic output
  )

  # Extract the answer from the API response
  answer = res.choices[0].message.content.strip()

  # Return the extracted answer
  return answer

Temperature = 0: When the temperature is set to 0, the model generates the most likely response according to its training data. This means it will always choose the highest probability token at each step, leading to more deterministic and predictable output. Essentially, the model will be very focused on producing the "safest" and most expected continuation of the text.

In [45]:
query="I'm allergic to Salicylic Acid, its safe to use the Blue Algae Vitamin C Skinclarity Brightening Serum?"
documents=retrieve(query,top_k=3,namespace='sephora', emb_model='text-embedding-3-small')
context_prompt=prompt_with_context_builder(query,documents)
answer=question_answering(context_prompt,documents,chat_model='gpt-3.5-turbo')

In [46]:
print(answer)

I'm sorry, but the Blue Algae Vitamin C Skinclarity Brightening Serum contains Salicylic Acid in its ingredients. Since you are allergic to Salicylic Acid, it may not be safe for you to use this serum.


In [ ]:
query="I'm allergic to Salicylic Acid, its safe to use the P474809?"
documents=retrieve(query,top_k=3,namespace='sephora', emb_model='text-embedding-3-small')
context_prompt=prompt_with_context_builder(query,documents)
answer=question_answering(context_prompt,documents,chat_model='gpt-3.5-turbo')

In [ ]:
print(answer)

I'm sorry, but the provided context does not include any information about product ID P474809. I recommend checking the product details or consulting with a healthcare professional to ensure it is safe for you to use based on your allergy to Salicylic Acid.


In [ ]:
query="what its the product_id of Blue Algae Vitamin C Skinclarity Brightening Serum"
documents=retrieve(query,top_k=3,namespace='sephora', emb_model='text-embedding-3-small')
context_prompt=prompt_with_context_builder(query,documents)
answer=question_answering(context_prompt,documents,chat_model='gpt-3.5-turbo')

In [ ]:
print(answer)

The product_id of Blue Algae Vitamin C Skinclarity Brightening Serum is P474809.


In [ ]:
query="what are all the coincident ingredients between Blue Algae Vitamin C Skinclarity Brightening Serum and Waterless Vitamin C Serum"
documents=retrieve(query,top_k=3,namespace='sephora', emb_model='text-embedding-3-small')
context_prompt=prompt_with_context_builder(query,documents)
answer=question_answering(context_prompt,documents,chat_model='gpt-3.5-turbo')

In [ ]:
print(answer)

The coincident ingredients between Blue Algae Vitamin C Skinclarity Brightening Serum and Waterless Vitamin C Serum are Ascorbic Acid and Sodium Citrate.


In [ ]:
query="what are all the ingredients of Blue Algae Vitamin C Skinclarity Brightening Serum"
documents=retrieve(query,top_k=3,namespace='sephora', emb_model='text-embedding-3-small')
context_prompt=prompt_with_context_builder(query,documents)
answer=question_answering(context_prompt,documents,chat_model='gpt-3.5-turbo')

In [ ]:
print(answer)

Water Aqua Eau, Caprylic/Capric Triglyceride, Propanediol, Glycerin, Butylene Glycol, Ascorbic Acid, Triheptanoin, Pseudoalteromonas Ferment Extract, C13-15 Alkane, Parachlorella Beijerinckii Exopolysaccharides, Biosaccharide Gum-1, Bakuchiol, Beta Vulgaris (Beet) Root Extract, Hydrolyzed Corn Starch, Sodium Hyaluronate, Melia Azadirachta Leaf Extract, Melia Azadirachta Flower Extract, Corallina Officinalis Extract, Coccinia Indica Fruit Extract, Solanum Melongena (Eggplant) Fruit Extract, Aloe Barbadensis Flower Extract, Simmondsia Chinensis (Jojoba) Seed Oil, Curcuma Longa (Turmeric) Root Extract, Ocimum Basilicum (Basil) Flower/Leaf Extract, Ocimum Sanctum Leaf Extract, Salicylic Acid, Tocopherol, Sodium Polyacryloyldimethyl Taurate, Sodium Citrate, Phycocyanin, Lavandula Angustifolia (Lavender) Oil, Tanacetum Vulgare Extract, Tetrasodium Glutamate Diacetate, Potassium Sorbate, Caprylyl Glycol, Ethylhexylglycerin, Phenoxyethanol, Sodium Salicylate, Linalool, Limonene.


In [ ]:
query="what it is a product of the same category, same ingredients, cheapear than Blue Algae Vitamin C Skinclarity Brightening Serum"
documents=retrieve(query,top_k=3,namespace='sephora', emb_model='text-embedding-3-small')
context_prompt=prompt_with_context_builder(query,documents)
answer=question_answering(context_prompt,documents,chat_model='gpt-3.5-turbo')

In [ ]:
print(answer)

The product "10 Vitamin C Brightening Serum" is of the same category, face serums, has similar ingredients, and is cheaper than the "Blue Algae Vitamin C Skinclarity Brightening Serum."


In [ ]:
query="I'm planning to buy Blue Algae Vitamin C Skinclarity Brightening Serum,Waterless Vitamin C Serum and GENIUS Sleeping Collagen Moisturizer, im allergict to salicylic acid, check if this products contain it and if so suggest me another similar products"
documents=retrieve(query,top_k=3,namespace='sephora', emb_model='text-embedding-3-small')
context_prompt=prompt_with_context_builder(query,documents)
answer=question_answering(context_prompt,documents,chat_model='gpt-3.5-turbo')

In [ ]:
print(answer)

The Blue Algae Vitamin C Skinclarity Brightening Serum does contain Salicylic Acid in its ingredients. The GENIUS Ultimate AntiAging Vitamin C Serum also contains Salicylic Acid. Since you are allergic to Salicylic Acid, I recommend avoiding these products. I suggest looking for alternative serums and moisturizers that do not contain Salicylic Acid in their ingredients.


In [ ]:
# Listas de ingredientes como cadenas de texto
ingredientes_1 = 'Water (Aqua, Eau), Caprylic/Capric Triglyceride, Propanediol, Glycerin, Butylene Glycol, Ascorbic Acid, Triheptanoin, Pseudoalteromonas Ferment Extract, C13-15 Alkane, Parachlorella Beijerinckii Exopolysaccharides, Biosaccharide Gum-1, Bakuchiol, Beta Vulgaris Root Extract, Hydrolyzed Corn Starch, Sodium Hyaluronate, Melia Azadirachta Leaf Extract, Melia Azadirachta Flower Extract, Corallina Officinalis Extract, Coccinia Indica Fruit Extract, Solanum Melongena Fruit Extract, Aloe Barbadensis Flower Extract, Simmondsia Chinensis (Jojoba) Seed Oil, Curcuma Longa Root Extract, Ocimum Basilicum Flower/Leaf Extract, Ocimum Sanctum Leaf Extract, Salicylic Acid, Tocopherol, Sodium Polyacryloyldimethyl Taurate, Sodium Citrate, Phycocyanin, Lavandula Angustifolia Oil, Tanacetum Vulgare Extract, Tetrasodium Glutamate Diacetate, Potassium Sorbate, Caprylyl Glycol, Ethylhexylglycerin, Phenoxyethanol, Sodium Salicylate, Linalool, Limonene.'
ingredientes_2 = 'Water Aqua Eau, Caprylic/Capric Triglyceride, Propanediol, Glycerin, Butylene Glycol, Ascorbic Acid, Triheptanoin, Pseudoalteromonas Ferment Extract, C13-15 Alkane, Parachlorella Beijerinckii Exopolysaccharides, Biosaccharide Gum-1, Bakuchiol, Beta Vulgaris (Beet) Root Extract, Hydrolyzed Corn Starch, Sodium Hyaluronate, Melia Azadirachta Leaf Extract, Melia Azadirachta Flower Extract, Corallina Officinalis Extract, Coccinia Indica Fruit Extract, Solanum Melongena (Eggplant) Fruit Extract, Aloe Barbadensis Flower Extract, Simmondsia Chinensis (Jojoba) Seed Oil, Curcuma Longa (Turmeric) Root Extract, Ocimum Basilicum (Basil) Flower/Leaf Extract, Ocimum Sanctum Leaf Extract, Salicylic Acid, Tocopherol, Sodium Polyacryloyldimethyl Taurate, Sodium Citrate, Phycocyanin, Lavandula Angustifolia (Lavender) Oil, Tanacetum Vulgare Extract, Tetrasodium Glutamate Diacetate, Potassium Sorbate, Caprylyl Glycol, Ethylhexylglycerin, Phenoxyethanol, Sodium Salicylate, Linalool, Limonene.'

# Convertir las cadenas en listas de ingredientes
lista_ingredientes_1 = [ingrediente.strip() for ingrediente in ingredientes_1.split(',')]
lista_ingredientes_2 = [ingrediente.strip() for ingrediente in ingredientes_2.split(',')]

# Calcular las coincidencias
coincidencias = set(lista_ingredientes_1) & set(lista_ingredientes_2)

# Calcular el porcentaje de coincidencias respecto a la primera lista
porcentaje_coincidencia = (len(coincidencias) / len(lista_ingredientes_1)) * 100

print(f'Porcentaje de coincidencia: {porcentaje_coincidencia:.2f}%')


Porcentaje de coincidencia: 82.93%


In [ ]:
query="which product of the Skincare primary category has the highest rating and why"
documents=retrieve(query,top_k=3,namespace='sephora', emb_model='text-embedding-3-small')
context_prompt=prompt_with_context_builder(query,documents)
answer=question_answering(context_prompt,documents,chat_model='gpt-3.5-turbo')

In [ ]:
print(answer)

The product with the highest rating in the Skincare primary category is "Beauty Sleep Power Peel AgeDefying Night Peel" with a rating of 4.6. This product has a higher rating compared to the other skincare products listed, making it the top-rated product in the Skincare category.


In [ ]:
#Liquid Gold Midnight Reboot Serum with 14% Glycolic Acid and Tripeptide-5
query="whats the rating of Liquid Gold Midnight Reboot Serum with 14% Glycolic Acid and Tripeptide-5 and which is its primary category?"
documents=retrieve(query,top_k=3,namespace='sephora', emb_model='text-embedding-3-small')
context_prompt=prompt_with_context_builder(query,documents)
answer=question_answering(context_prompt,documents,chat_model='gpt-3.5-turbo')

In [ ]:
print(answer)

The rating of Liquid Gold Midnight Reboot Serum with 14% Glycolic Acid and Tripeptide-5 is 5.0, and its primary category is Skincare.


In [ ]:
#pruebas
query="Blue Algae Vitamin C Skinclarity Brightening Serum"
query_response=client.embeddings.create(
    input=query,
    model='text-embedding-3-small')
query_emb=query_response.data[0].embedding
retrieved_docs=index.query(vector=query_emb,
                           top_k=5,
                           namespace=namespace,
                           include_metadata=True)


In [ ]:
for results in retrieved_docs['matches']:
  print(f"{round(results['score'], 2)}:{results['metadata']['content']}")

0.77:ID:(P474809), Name:(Blue Algae Vitamin C Skinclarity Brightening Serum), highlights:(sin info), primary_category:(Skincare), secondary_category:(Treatments), tertiary_category:(Face Serums), rating:(4.6087), size:(1 oz 30 mL), price:(72.0), ingredients:(Water Aqua Eau CaprylicCapric Triglyceride Propanediol Glycerin Butylene Glycol Ascorbic Acid Triheptanoin Pseudoalteromonas Ferment Extract C1315 Alkane Parachlorella Beijerinckii Exopolysaccharides Biosaccharide Gum1 Bakuchiol Beta Vulgaris Root Extract Hydrolyzed Corn Starch Sodium Hyaluronate Melia Azadirachta Leaf Extract Melia Azadirachta Flower Extract Corallina Officinalis Extract Coccinia Indica Fruit Extract Solanum Melongena Fruit Extract Aloe Barbadensis Flower Extract Simmondsia Chinensis Jojoba Seed Oil Curcuma Longa Root Extract Ocimum Basilicum FlowerLeaf Extract Ocimum Sanctum Leaf Extract Salicylic Acid Tocopherol Sodium Polyacryloyldimethyl Taurate Sodium Citrate Phycocyanin Lavandula Angustifolia Oil Tanacetum V